In [1]:
import os
import asyncio
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout

import time

In [2]:
SEASONS = list(range(2023, 2025))

In [3]:
SEASONS

[2023, 2024]

In [4]:
DATA_DIR = "Data"
STANDINGS_DIR = os.path.join(DATA_DIR, "Standings")
SCORES_DIR = os.path.join(DATA_DIR, "Scores")

os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

In [5]:
async def get_html(url, selector, sleep=5, retries=6):
    html = None
    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=True)
        page = await browser.new_page()
        for i in range(1, retries+1):
            try:
                await page.goto(url, timeout=30000)
                print(await page.title())
                html = await page.inner_html(selector, timeout=30000)
            except PlaywrightTimeout as e:
                print(f"Timeout error on {url}: {str(e)}")
                await asyncio.sleep(sleep * i)
            else:
                break
        await page.close()
        await browser.close()
    return html

In [6]:
season = 2023

url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"


In [7]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all("a") 
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue 
            
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)
    

In [9]:
async def scrape_game (standings_files):
    with open (standings_files, 'r') as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/") [-1])
        if os.path.exists(save_path):
            continue 

        html = await get_html(url, "#content")
        if not html:
            continue 
        with open(save_path, "w+") as f:
            f.write(html)
    

In [10]:
#for f in standings_files:
    #filepath = os.path.join(STANDINGS_DIR, f)
    
    #await scrape_game(filepath)

In [12]:
for season in SEASONS:
    await scrape_season(season)

Timeout error on https://www.basketball-reference.com/leagues/NBA_2023_games.html: Timeout 30000ms exceeded.
=========================== logs ===========================
navigating to "https://www.basketball-reference.com/leagues/NBA_2023_games.html", waiting until "load"
Timeout error on https://www.basketball-reference.com/leagues/NBA_2023_games.html: Timeout 30000ms exceeded.
=========================== logs ===========================
navigating to "https://www.basketball-reference.com/leagues/NBA_2023_games.html", waiting until "load"
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
Timeout error on https://basketball-reference.com/leagues/NBA_2023_games-march.html: Timeout 30000ms exceeded.
=========================== logs ======================

In [13]:
standings_files = os.listdir(STANDINGS_DIR)

In [14]:
standings_files

['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2023_games-may.html',
 'NBA_2019_games-april.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2023_games-january.html',
 'NBA_2016_games-april.html',
 'NBA_2024_games-march.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2024_games-november.html',
 'NBA_2017_games-february.html',
 'NBA_2024_games-december.html',
 'NBA_2016_games-february.html',
 'NBA_2017_games-october.html',
 'NBA_2018_games-april.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NBA_2021_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2020_games-october-2019.html',
 'NBA_2022_games-december.html',
 'NBA_2023_games-novemb